# Create database for unigrams (later for bigrams)

In [45]:
import pandas as pd
import sqlite3
import os

In [54]:
os.chdir("/Users/nikolaevaanna/Downloads/unigram_result_dates")
paths = [i for i in os.listdir(".") if not i.endswith(".db")]

In [55]:
paths

['unigram-result-dates_j.tsv',
 'unigram-result-dates_k.tsv',
 'unigram-result-dates_i.tsv',
 'unigram-result-dates_h.tsv',
 'unigram-result-dates_l.tsv',
 'unigram-result-dates_m.tsv',
 'unigram-result-dates_z.tsv',
 'unigram-result-dates_x.tsv',
 'unigram-result-dates_o.tsv',
 'unigram-result-dates_n.tsv',
 'unigram-result-dates_t.tsv',
 'unigram-result-dates_c.tsv',
 'unigram-result-dates_b.tsv',
 'unigram-result-dates_u.tsv',
 'unigram-result-dates_a.tsv',
 'unigram-result-dates_v.tsv',
 'unigram-result-dates_e.tsv',
 'unigram-result-dates_r.tsv',
 'unigram-result-dates_s.tsv',
 'unigram-result-dates_d.tsv',
 'unigram-result-dates_f.tsv',
 'unigram-result-dates_p.tsv',
 'unigram-result-dates_g.tsv']

In [49]:
conn = sqlite3.connect("wikidata.db")
cursor = conn.cursor()

In [ ]:
cursor.execute("""CREATE TABLE ngrams
                  (ngram text, 
                  start_letter text,
                  Q_number text, 
                  wiki_entity text, 
                  property_code text,
                  property_value text,
                  object text,
                  organization text,
                  just_date text,
                  start_time text,
                  end_time text,
                  time_point text,
                  growth_speed real,
                  google_year_1 integer,
                  google_year_2 integer)
               """)

In [ ]:
conn.commit()

In [56]:
def update_wikidata_database(path):
    
    dataframe = pd.read_csv(path, sep="\t") 
    
    for i in dataframe.index:
        years = dataframe.loc[i, "Year or Period"]
        if years.count(",") == 1:
            years = years.strip("[]").split(", ")
            dataframe.loc[i, "year1"] = int(years[0])
            dataframe.loc[i, "year2"] = int(years[1])
        else:
            dataframe.loc[i, "year1"] = int(years)
            
    start_letters = [i[0] for i in dataframe.iloc[:, 1]]
    dataframe.insert(2, 'start_letter', start_letters)
    
    df_list = dataframe.values.tolist()
    new_df_list = []
    for line in df_list:
        new_line = tuple(line[1:])
        new_df_list.append(new_line)
        
    cursor.executemany("INSERT INTO ngrams VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", new_df_list)
    conn.commit()
    
    return "Ngrams from " + path + " added into wikidata.db"

In [57]:
for i in paths:
    update_wikidata_database(i)

In [ ]:
# than I also added index on start_letter column